In [1]:
# トレーニングに用いるデータのスクレイピングjockey_idから過去データ(近走成績,年度別成績)を取る 当日も取れる情報がよい
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import time
from tqdm import tqdm as tqdm
import numpy as np
pd.set_option('display.max_columns', 100)

In [29]:
def sclape_jockey_results(jockey_id_list, pre_jockey_results_thisyear={}, pre_jockey_results_allyear={}):
    jockey_results_thisyear = pre_jockey_results_thisyear
    jockey_results_allyear = pre_jockey_results_allyear
    for jockey_id in tqdm(jockey_id_list):
        if jockey_id in jockey_results_allyear.keys():
            continue
        try:
            url = 'https://db.netkeiba.com/jockey/' + jockey_id
            #近走
            df = pd.read_html(url)[0]


            url2 = 'https://db.netkeiba.com/jockey/result/' + jockey_id
            #年度
            df2 = pd.read_html(url2, header=0)[0]

            df2.rename(columns={'重賞':'重賞_出走','重賞.1':'重賞_勝利','平場':'平場_出走','平場.1':'平場_勝利','ダート':'ダート_出走','ダート.1':'ダート_勝利'}, inplace=True)
            df2.rename(columns={'特別':'特別_出走','特別.1':'特別_勝利','芝':'芝_出走','芝.1':'芝_勝利'}, inplace=True)
            df2.drop(df2.index[[0]], inplace=True)
            
            jockey_results_thisyear[jockey_id] = df
            jockey_results_allyear[jockey_id] = df2

            time.sleep(0.1)
        except IndexError:
            print("IndexError")
            continue
        except:
            # for文から抜け出す
            print("bleak")
            break
    
    for key in tqdm(jockey_results_thisyear.keys()):
        jockey_results_thisyear[key].index = [key] * len(jockey_results_thisyear[key])

    jockey_results_thisyear = pd.concat([jockey_results_thisyear[key] for key in tqdm(jockey_results_thisyear.keys())], sort=False)

    for key in tqdm(jockey_results_allyear.keys()):
        jockey_results_allyear[key].index = [key] * len(jockey_results_allyear[key])

    jockey_results_allyear = pd.concat([jockey_results_allyear[key] for key in tqdm(jockey_results_allyear.keys())], sort=False)

    return jockey_results_thisyear, jockey_results_allyear

In [7]:
jockey_id = '05339'
url = 'https://db.netkeiba.com/jockey/' + jockey_id
#近走
df = pd.read_html(url)[0]
df

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,単勝,人気,着順,馬名,斤量,距離,馬場,タイム,着差,通過,ペース,上り,馬体重,勝ち馬,賞金(万円)
0,2020/09/13,4中山2,曇,11,京成杯オータムH(G3),NaN,16,3,5,3.9,1,10,アンドラステ,53,芝1600,良,1:34.5,0.6,6-9-13,35.0-35.6,35.2,470(+2),トロワゼトワル,NaN
1,2020/09/13,4中山2,曇,10,初風S(1600万下),NaN,15,4,6,3.0,1,6,ノンライセンス,53,ダ1200,良,1:10.9,1.0,5-5,33.9-36.0,36.7,466(+4),ロードラズライト,NaN
2,2020/09/13,4中山2,曇,9,白井特別(1000万下),NaN,9,4,4,2.7,2,4,エリスライト,55,芝1800,良,1:50.0,0.4,6-6-6-6,37.8-35.5,35.4,460(+6),デクレアラー,230.0
3,2020/09/13,4中山2,曇,8,3歳以上1勝クラス,NaN,14,8,14,2.6,1,5,ダイシンウィット,55,ダ1200,良,1:12.3,1.1,3-2,34.4-36.8,37.8,496(+4),ブランオラージュ,76.0
4,2020/09/13,4中山2,曇,7,3歳以上1勝クラス,NaN,16,2,3,5.4,2,9,ライフレッスンズ,55,芝1600,良,1:36.0,1.0,12-14-14,35.0-36.1,35.3,506(0),フォースオブウィル,NaN
5,2020/09/13,4中山2,曇,6,3歳以上1勝クラス,NaN,10,4,4,2.0,1,1,ピオノノ,54,ダ1800,良,1:53.0,-0.1,2-2-2-2,37.5-38.1,38.0,514(+2),(アポロティアモ),760.0
6,2020/09/13,4中山2,曇,4,3歳以上1勝クラス,NaN,11,7,8,2.2,1,2,フジノタカネ,57,ダ1200,稍,1:11.6,0.1,4-4,33.9-37.6,36.7,524(-4),インウィクトス,300.0
7,2020/09/13,4中山2,曇,3,2歳新馬,NaN,15,8,16,7.8,4,2,レッドウッドヒル,54,ダ1800,稍,1:56.2,0.0,6-5-6-4,38.2-38.7,38.2,488(0),ティアップリオン,280.0
8,2020/09/13,4中山2,曇,2,2歳未勝利,NaN,11,8,10,1.2,1,4,グアドループ,54,芝2000,良,2:03.9,0.2,1-2-2-2,37.4-36.5,36.7,514(+4),アドマイヤハレー,77.0
9,2020/09/13,4中山2,曇,1,2歳未勝利,NaN,12,8,12,6.0,3,1,リルブラーヴ,54,ダ1200,稍,1:13.4,-0.3,2-2,34.4-39.0,38.9,476(+6),(バルネージュ),510.0


In [17]:
url2 = 'https://db.netkeiba.com/jockey/result/' + jockey_id
#年度
df2 = pd.read_html(url2, header=0)[0]
df2.rename(columns={'重賞':'重賞_出走','重賞.1':'重賞_勝利','平場':'平場_出走','平場.1':'平場_勝利','ダート':'ダート_出走','ダート.1':'ダート_勝利'}, inplace=True)
df2.rename(columns={'特別':'特別_出走','特別.1':'特別_勝利','芝':'芝_出走','芝.1':'芝_勝利'}, inplace=True)
df2.drop(df2.index[[0]], inplace=True)
df2

,年度,順位,1着,2着,3着,着外,重賞_出走,重賞_勝利,特別_出走,特別_勝利,平場_出走,平場_勝利,芝_出走,芝_勝利,ダート_出走,ダート_勝利,勝率,連対率,複勝率,収得賞金(万円),代表馬
1,累計,NaN,1256,911,710,3198,521,98,1610,318,3944,840,3487,765,2588,491,.207,.357,.474,2726274.9,NaN
2,2020,1,135,85,62,239,38,8,142,29,341,98,298,69,223,66,.259,.422,.541,244691.4,アーモンドアイ
3,2019,1,164,123,78,285,58,16,178,37,414,111,392,107,258,57,.252,.442,.562,354723.8,グランアレグリア
4,2018,1,215,135,98,324,68,20,201,52,503,143,463,137,309,78,.278,.453,.580,466023.5,アーモンドアイ
5,2017,1,199,138,106,366,70,14,211,54,528,131,469,127,340,72,.246,.417,.548,414622.0,レイデオロ
6,2016,2,186,109,100,391,72,13,215,49,499,124,456,118,330,68,.237,.375,.503,404074.8,サトノダイヤモンド
7,2015,4,112,90,70,301,52,9,170,31,351,72,358,72,215,40,.195,.353,.475,223069.4,ノンコノユメ
8,2014,50,22,16,19,87,11,2,34,1,99,19,75,8,69,14,.153,.264,.396,42533.2,サトノノブレス
9,2013,42,29,23,17,102,18,5,45,8,108,16,90,12,81,17,.170,.304,.404,70880.2,ベルシャザール
10,2012,48,23,20,15,94,17,2,42,8,93,13,91,16,61,7,.151,.283,.382,54974.0,オーシャンブルー


In [30]:
jockey_id_list = ['01151', '00700', '01091']
test1, test2 = sclape_jockey_results(jockey_id_list)
test2

100%|██████████| 3/3 [00:00<00:00, 19152.07it/s]


,年度,順位,1着,2着,3着,着外,重賞_出走,重賞_勝利,特別_出走,特別_勝利,平場_出走,平場_勝利,芝_出走,芝_勝利,ダート_出走,ダート_勝利,勝率,連対率,複勝率,収得賞金(万円),代表馬
01151,累計,NaN,66,56,96,1961,4,0,151,3,2024,63,877,19,1302,47,.030,.056,.100,90900.7,NaN
01151,2020,125,1,0,1,40,0,0,1,0,41,1,17,0,25,1,.024,.024,.048,911.0,カッティングロック
01151,2019,119,3,0,4,68,0,0,12,1,63,2,33,1,42,2,.040,.040,.093,3640.5,サウンドマジック
01151,2018,104,5,4,7,179,1,0,24,0,170,5,86,1,109,4,.026,.046,.082,7130.2,ハウエバー
01151,2017,67,13,11,21,367,0,0,33,1,379,12,153,4,259,9,.032,.058,.109,17048.2,カレン
01151,2016,67,14,14,19,454,1,0,37,0,463,14,202,1,299,13,.028,.056,.094,20064.6,トウショウカウント
01151,2015,50,23,20,35,531,2,0,36,1,571,22,255,11,354,12,.038,.071,.128,32295.2,マリカ
01151,2014,91,7,7,9,322,0,0,8,0,337,7,131,1,214,6,.020,.041,.067,9811.0,トーセンプリモ
00700,累計,NaN,1586,1420,1244,9669,1087,76,4029,426,8803,1084,7595,848,6324,738,.114,.216,.305,3321064.6,NaN
00700,2020,100,3,4,3,36,2,0,17,1,27,2,25,1,21,2,.065,.152,.217,7915.3,ヒルノサルバドール


In [31]:
horse_results = pd.read_pickle('1-2results2015_2020.pickle')
print(len(horse_results))
len(horse_results['jockey_id'].unique())

223298


274

In [32]:
jockey_id_list = horse_results['jockey_id'].unique()
len(jockey_id_list)

274

In [33]:
jockey_results_thisyear2015_2020, jockey_results_allyear2015_2020 = sclape_jockey_results(jockey_id_list)

100%|██████████| 274/274 [00:00<00:00, 694404.41it/s]


In [34]:
jockey_results_thisyear2015_2020.head()

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,単勝,人気,着順,馬名,斤量,距離,馬場,タイム,着差,通過,ペース,上り,馬体重,勝ち馬,賞金(万円)
01151,2020/09/12,4中山1,雨,6,3歳以上1勝クラス,NaN,15.0,7.0,13,145.5,15.0,8,タンレイ,55.0,ダ1200,稍,1:12.2,0.5,2-2,33.8-37.9,38.3,438(-6),ロマンスマジック,NaN
01151,2020/08/16,3新潟2,曇,3,2歳未勝利,NaN,18.0,5.0,9,307.4,17.0,18,ホタルマル,54.0,芝1200,稍,1:13.8,3.6,18-18,34.4-35.8,36.7,472(+12),ロンギングバース,NaN
01151,2020/08/15,3新潟1,曇,12,3歳以上1勝クラス,NaN,15.0,3.0,4,52.6,12.0,7,カッティングロック,54.0,ダ1800,良,1:54.8,0.9,15-15-14-14,37.6-38.4,37.4,514(+8),ブルーノジュネス,NaN
01151,2020/08/15,3新潟1,曇,3,3歳未勝利,NaN,18.0,6.0,12,167.5,14.0,15,クリススキャット,54.0,芝2000,良,2:06.0,2.4,2-2-2-2,36.9-36.6,38.9,492(0),ローズパルファン,NaN
01151,2020/07/19,2福島6,晴,12,3歳以上1勝クラス,NaN,15.0,4.0,6,28.6,9.0,11,トライゲッター,57.0,ダ1700,良,1:47.6,1.4,7-7-9-10,28.7-40.0,40.4,492(-2),ファイアランス,NaN


In [35]:
jockey_results_allyear2015_2020.head()

,年度,順位,1着,2着,3着,着外,重賞_出走,重賞_勝利,特別_出走,特別_勝利,平場_出走,平場_勝利,芝_出走,芝_勝利,ダート_出走,ダート_勝利,勝率,連対率,複勝率,収得賞金(万円),代表馬
01151,累計,NaN,66,56,96,1961,4,0,151,3,2024,63,877,19,1302,47,.030,.056,.100,90900.7,NaN
01151,2020,125,1,0,1,40,0,0,1,0,41,1,17,0,25,1,.024,.024,.048,911.0,カッティングロック
01151,2019,119,3,0,4,68,0,0,12,1,63,2,33,1,42,2,.040,.040,.093,3640.5,サウンドマジック
01151,2018,104,5,4,7,179,1,0,24,0,170,5,86,1,109,4,.026,.046,.082,7130.2,ハウエバー
01151,2017,67,13,11,21,367,0,0,33,1,379,12,153,4,259,9,.032,.058,.109,17048.2,カレン


In [36]:
jockey_results_thisyear2015_2020.to_pickle('1-8jockey_results_thisyear2015-2020.pickle')
jockey_results_allyear2015_2020.to_pickle('1-8jockey_results_allyear2015-2020.pickle')